<h1>It is a Coursera Capstone project:</h1>

In [1]:
print('Hello Capstone Project Course!') 

Hello Capstone Project Course!


<h1>Postalcodes from Wikipedia</h1>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url='https://en.m.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

res = requests.get(url)
res = BeautifulSoup(res.content,'lxml')
table = res.find('table')
df=pd.read_html(str(table))[0]

In [ ]:
df.columns=['PostalCode', 'Borough', 'Neighborhood'] 
df=df[df['Borough']!='Not assigned']
df.loc[df['Neighborhood']=='Not assigned','Neighborhood']=df['Borough']
df['Neighborhood']=df['Neighborhood'].str.replace(' /', ',')
df.reset_index(drop=True, inplace=True)

In [ ]:
print(df.head())
df.shape

  PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


(103, 3)

<h1>Geolocation data</h1>

In [ ]:
locdf=pd.read_csv('https://cocl.us/Geospatial_data')
locdf.columns=['PostalCode', 'Latitude', 'Longitude']
df=df.join(locdf.set_index('PostalCode'), on='PostalCode')

print(df.head())

  PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  


<h1>Foursquare Venues</h1>

In [ ]:
filter=[('Toronto' in bor)  for bor in df['Borough']]
toronto=df[filter].reset_index(drop=True)
print(toronto.head())
print(toronto.shape)

  PostalCode           Borough                                 Neighborhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031  
(39, 5)


In [ ]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: - 

In [ ]:
address='Toronto'
geolocator=Nominatim(user_agent='tor_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude, longitude)

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [ ]:
map=folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lon, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
  label=folium.Popup(label, parse_html=True)
  folium.features.CircleMarker(
      [lat, lon], 
      radius=5,
      color='blue',
      popup=label,
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map)

map

In [ ]:
CLIENT_ID='YOUR_ID'
CLIENT_SECRET='YOUR_SECRET'
VERSION='20180605'
LIMIT=100
RADIUS=500

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
def getNearbyVenues(names, lats, lons):
    list=[]
    for n, la, lo in zip(names, lats, lons):
        url= 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, la, lo, RADIUS, LIMIT)
        results= requests.get(url).json()['response']['groups'][0]['items']
        list.append([(
         n, 
         la, 
         lo, 
         v['venue']['name'], 
         v['venue']['location']['lat'], 
         v['venue']['location']['lng'], 
         v['venue']['categories'][0]['name']) for v in results]
        )
    
    nearby=pd.DataFrame([item for vlist in list for item in vlist])
    nearby.columns=[
     'Neighborhood',
     'Neighborhood Latitude', 
     'Neighborhood Longitude',
     'Venue',
     'Venue Latitude',
     'Venue Longitude', 
     'Venue Category'
    ]

    return(nearby)

In [ ]:
venues=getNearbyVenues(
  names=toronto['Neighborhood'],
  lats=toronto['Latitude'],
  lons=toronto['Longitude']
)
print(venues.shape)

In [ ]:
onehot=pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot['Neighborhood']=venues['Neighborhood']
fixed_columns=[onehot.columns[-1]]+list(onehot.columns[:-1])
onehot=onehot[fixed_columns]
grouped=onehot.groupby('Neighborhood').mean().reset_index()
print(grouped.shape)

In [ ]:
def most_common_venues(row, top):
    row_categories=row.iloc[1:]
    row_categories_sorted= row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:top]

In [ ]:
import numpy as np

In [ ]:
top=10
indicators=['st', 'nd', 'rd']
columns=['Neighborhood']
for ind in np.arange(top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicator[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

sorted=pd.DataFrame(columns=columns)
sorted['Neighborhood']=grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    sorted.iloc[ind, 1:]=most_common_venues(grouped.iloc[ind, :],top)

print(sorted.head())

<h1>Clustering</h1>

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
clustering=grouped.drop('Neighborhood', 1)

k=5
kmeans=KMeans(n_clusters=k, random_state=0)
kmeans.fit(clustering)

sorted.insert(0,'Cluster Labels', kmeans.labels_)

merged=toronto
merged=merged.join(sorted.set_index('Neighborhood'), on='Neighborh

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
map=folium.Map(location=[latitude, longitude], zoom_start=12)

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]

for lat, lon, poi, clus in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label=folium.Popup(str(poi)+' Cluster '+str(clus), parse_html=True)
    folium.features.CircleMarker(
        [lat, lon], 
        radius=5,
        color=rainbow[clus-1], 
        popup=label,
        fill=True,
        fill_color=rainbow[clus-1], 
        fill_opacity=0.7).add_to(map)
                                                   
map

<h2>Cluster 1</h2>

In [ ]:
merged.loc[merged['Cluster Labels']==0, merged.columns[[2]+list(range(k, merged.shape[1]))]]

<h2>Cluster 2</h2>

In [ ]:
merged.loc[merged['Cluster Labels']==1, merged.columns[[2]+list(range(k, merged.shape[1]))]]

<h2>Cluster 3</h2>

In [ ]:
merged.loc[merged['Cluster Labels']==2, merged.columns[[2]+list(range(k, merged.shape[1]))]]

<h2>Cluster 4</h2>

In [ ]:
merged.loc[merged['Cluster Labels']==3, merged.columns[[2]+list(range(k, merged.shape[1]))]]

<h2>Cluster 5</h2>

In [ ]:
merged.loc[merged['Cluster Labels']==4, merged.columns[[2]+list(range(k, merged.shape[1]))]]